In [61]:
#
#Loading necessary modules
#

import os
from sklearn.ensemble import RandomForestClassifier as RF_SK
from sklearn.model_selection import train_test_split as train_test_split_sk
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.externals.joblib import parallel_backend
sk_rf = RF_SK(n_jobs=-1, n_estimators=50)
root = os.getcwd
project_hpc = os.path.join(root, '../'
import matplotlib.pyplot as plt
# data_hpc_protocol = os.path.join(project_hpc,"PAMAP2_Dataset/Protocol")
data_hpc_protocol = os.path.join(project_hpc,"input_data")
data_hpc_protocol_csv = os.path.join(project_hpc,"input_data")
# data_hpc_protocol_csv = os.path.join(data_hpc_protocol,'csv_files')
# data_hpc_protocol_cleaned_csv = os.path.join(project_hpc, 'input_data/combined_patient_csv_file_cleaned.csv')

In [2]:
#
#Setting up dask cluster environment
#
#
from dask.distributed import Client,progress, wait
client = Client('172.16.0.70:8786')


In [31]:
%load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


In [3]:
client

Client Scheduler: tcp://172.16.0.70:8786 Dashboard: http://172.16.0.70:8787/status,Cluster Workers: 4 Cores: 192 Memory: 539.65 GB


# Reading data

In [10]:
%%time
#
#Reading patients data csv files using pandas dataframe:
#

import dask.dataframe as df

for idx, patient_file in enumerate(os.listdir(data_hpc_protocol_csv)):
    if idx == 0:
        df = pd.read_csv(os.path.join(data_hpc_protocol_csv,patient_file)
                                                                  , header = None)
        patient_all_pd = df
    else:
        df = pd.read_csv(os.path.join(data_hpc_protocol_csv,patient_file)
                                                                  , header = None)
        patient_all_pd = patient_all_pd.append(df)

# patient_all_pd.to_csv(os.path.join(root,'input_data', 'combined_patient_csv_file'))
     

CPU times: user 1min 51s, sys: 14.7 s, total: 2min 6s
Wall time: 22.9 s


In [12]:
patient_all_pd.shape

(2872533, 54)

In [13]:
#
#Manipulating column names
#
imus = [str('imu_' + str(x)) for x in list(range(51))]
col_names = ['time_stamp', 'activity_id', 'heart_rate_bpm']
col_names = col_names + imus
# patient_all_pd.columns = col_names

In [22]:
col_names

['time_stamp',
 'activity_id',
 'heart_rate_bpm',
 'imu_0',
 'imu_1',
 'imu_2',
 'imu_3',
 'imu_4',
 'imu_5',
 'imu_6',
 'imu_7',
 'imu_8',
 'imu_9',
 'imu_10',
 'imu_11',
 'imu_12',
 'imu_13',
 'imu_14',
 'imu_15',
 'imu_16',
 'imu_17',
 'imu_18',
 'imu_19',
 'imu_20',
 'imu_21',
 'imu_22',
 'imu_23',
 'imu_24',
 'imu_25',
 'imu_26',
 'imu_27',
 'imu_28',
 'imu_29',
 'imu_30',
 'imu_31',
 'imu_32',
 'imu_33',
 'imu_34',
 'imu_35',
 'imu_36',
 'imu_37',
 'imu_38',
 'imu_39',
 'imu_40',
 'imu_41',
 'imu_42',
 'imu_43',
 'imu_44',
 'imu_45',
 'imu_46',
 'imu_47',
 'imu_48',
 'imu_49',
 'imu_50']

In [23]:
%%time
#
#Reading data using dask's dataframe
#

import dask.dataframe as dd

patient_all_df = dd.read_csv(os.path.join(data_hpc_protocol_csv, '*.csv'),
                  storage_options={'anon': True})
patient_all_df.columns = col_names
progress(patient_all_df)

CPU times: user 96.2 ms, sys: 2.83 ms, total: 99.1 ms
Wall time: 102 ms


In [14]:
patient_all_pd.head()

,time_stamp,activity_id,heart_rate_bpm,imu_0,imu_1,imu_2,imu_3,imu_4,imu_5,imu_6,...,imu_41,imu_42,imu_43,imu_44,imu_45,imu_46,imu_47,imu_48,imu_49,imu_50
0,5.71,0,NaN,31.75,-1.54309,5.78581,7.73342,-1.58707,5.77449,7.89554,...,0.024932,-0.033188,0.018301,-46.7519,19.7154,9.44721,0.420604,0.506287,0.601928,0.452159
1,5.72,0,NaN,31.75,-1.69986,5.55834,7.61638,-1.57213,5.75928,7.89556,...,-0.015505,0.036956,-0.014232,-46.5148,20.3460,9.02181,0.424410,0.504851,0.601741,0.450455
2,5.73,0,NaN,31.75,-1.54603,5.55784,7.69566,-1.57276,5.72904,7.85033,...,0.019024,0.022195,0.010928,-46.4840,19.1874,9.59060,0.421472,0.506134,0.601398,0.452228
3,5.74,0,NaN,31.75,-1.57954,5.59708,7.81049,-1.51222,5.75881,7.85020,...,0.012959,-0.004018,-0.026358,-47.5172,19.6244,9.30279,0.421292,0.506310,0.599558,0.454636
4,5.75,0,NaN,31.75,-1.51225,5.55654,7.58071,-1.46764,5.72824,7.80492,...,0.073998,-0.041724,-0.006069,-47.5133,19.8842,10.15350,0.419774,0.508406,0.597869,0.455923


# Analyzing heart rate data 

In [53]:
heart_rate_df = patient_all_pd[['activity_id', 'heart_rate_bpm']]

In [54]:
heart_rate_df.head()

,activity_id,heart_rate_bpm
0,0,NaN
1,0,NaN
2,0,NaN
3,0,NaN
4,0,NaN


# Preprocessing the data

In [49]:
%%time
#
#Removing the time stamp colums and heart_rate_bpm column wih many missing values missing values 
#as they are very small amount compared to the size of the data
#and won't have have any effect on the performance of training model
#
patient_all_pd.drop(columns = ['time_stamp','heart_rate_bpm'], axis =1, inplace = True)
patient_all_pd.dropna(inplace = True)


#Storing the cleaned dataset in csv file
patient_all_pd.to_csv(os.path.join(root,'input_data', 'combined_patient_csv_file_cleaned.csv'), index=False)
# del patient_all_pd

CPU times: user 2min 47s, sys: 867 ms, total: 2min 48s
Wall time: 2min 59s


In [37]:
patient_all_pd.head()

,activity_id,imu_0,imu_1,imu_2,imu_3,imu_4,imu_5,imu_6,imu_7,imu_8,...,imu_41,imu_42,imu_43,imu_44,imu_45,imu_46,imu_47,imu_48,imu_49,imu_50
0,0,31.75,-1.54309,5.78581,7.73342,-1.58707,5.77449,7.89554,-0.028974,0.047977,...,0.024932,-0.033188,0.018301,-46.7519,19.7154,9.44721,0.420604,0.506287,0.601928,0.452159
1,0,31.75,-1.69986,5.55834,7.61638,-1.57213,5.75928,7.89556,0.000358,0.026022,...,-0.015505,0.036956,-0.014232,-46.5148,20.3460,9.02181,0.424410,0.504851,0.601741,0.450455
2,0,31.75,-1.54603,5.55784,7.69566,-1.57276,5.72904,7.85033,0.017085,0.011208,...,0.019024,0.022195,0.010928,-46.4840,19.1874,9.59060,0.421472,0.506134,0.601398,0.452228
3,0,31.75,-1.57954,5.59708,7.81049,-1.51222,5.75881,7.85020,0.013649,0.008197,...,0.012959,-0.004018,-0.026358,-47.5172,19.6244,9.30279,0.421292,0.506310,0.599558,0.454636
4,0,31.75,-1.51225,5.55654,7.58071,-1.46764,5.72824,7.80492,-0.014763,-0.006078,...,0.073998,-0.041724,-0.006069,-47.5133,19.8842,10.15350,0.419774,0.508406,0.597869,0.455923


In [50]:
%%time
#
#Reading cleaned data using pandas
#

patient_all_pd = pd.read_csv(os.path.join(root,'input_data', 'combined_patient_csv_file_cleaned.csv'))
columns  = patient_all_pd.columns

CPU times: user 22 s, sys: 2.09 s, total: 24.1 s
Wall time: 22 s


In [13]:
patient_all_pd.head()

,activity_id,imu_0,imu_1,imu_2,imu_3,imu_4,imu_5,imu_6,imu_7,imu_8,...,imu_41,imu_42,imu_43,imu_44,imu_45,imu_46,imu_47,imu_48,imu_49,imu_50
0,0,31.75,-1.54309,5.78581,7.73342,-1.58707,5.77449,7.89554,-0.028974,0.047977,...,0.024932,-0.033188,0.018301,-46.7519,19.7154,9.44721,0.420604,0.506287,0.601928,0.452159
1,0,31.75,-1.69986,5.55834,7.61638,-1.57213,5.75928,7.89556,0.000358,0.026022,...,-0.015505,0.036956,-0.014232,-46.5148,20.3460,9.02181,0.424410,0.504851,0.601741,0.450455
2,0,31.75,-1.54603,5.55784,7.69566,-1.57276,5.72904,7.85033,0.017085,0.011208,...,0.019024,0.022195,0.010928,-46.4840,19.1874,9.59060,0.421472,0.506134,0.601398,0.452228
3,0,31.75,-1.57954,5.59708,7.81049,-1.51222,5.75881,7.85020,0.013648,0.008197,...,0.012959,-0.004018,-0.026358,-47.5172,19.6244,9.30279,0.421292,0.506310,0.599558,0.454636
4,0,31.75,-1.51225,5.55654,7.58071,-1.46764,5.72824,7.80492,-0.014764,-0.006078,...,0.073998,-0.041724,-0.006069,-47.5133,19.8842,10.15350,0.419774,0.508406,0.597869,0.455923


In [51]:
patient_all_pd.shape

(2844868, 52)

In [6]:
%%time
#
#Reading data using dask's dataframe
#Doing collection = False will not load the data as Dask data frame, but as a bunch of pandas dataframe that will be located 
#in some part of the cluster 

import dask.dataframe as dd
patient_all_df = dd.read_csv(data_hpc_protocol_cleaned_csv, collection = False)

patient_all_df = client.persist(patient_all_df)

progress(patient_all_df)

CPU times: user 174 ms, sys: 16.5 ms, total: 191 ms
Wall time: 262 ms


In [66]:
%%time
#
#Reading data using dask's dataframe
#Doing collection = False will not load the data as Dask data frame, but as a bunch of pandas dataframe that will be located 
#in some part of the cluster 

import dask.dataframe as dd
patient_all_df = dd.from_pandas(patient_all_pd, npartitions = 10)
patient_all_df = patient_all_df.persist()
progress(patient_all_df)

CPU times: user 1.66 s, sys: 109 ms, total: 1.77 s
Wall time: 1.77 s


In [44]:
# client.cancel(patient_all_df)
# del patient_all_df

In [67]:
len(patient_all_df)

2844868

In [68]:
patient_all_df[:5]

,activity_id,imu_0,imu_1,imu_2,imu_3,imu_4,imu_5,imu_6,imu_7,imu_8,imu_9,imu_10,imu_11,imu_12,imu_13,imu_14,imu_15,imu_16,imu_17,imu_18,imu_19,imu_20,imu_21,imu_22,imu_23,imu_24,imu_25,imu_26,imu_27,imu_28,imu_29,imu_30,imu_31,imu_32,imu_33,imu_34,imu_35,imu_36,imu_37,imu_38,imu_39,imu_40,imu_41,imu_42,imu_43,imu_44,imu_45,imu_46,imu_47,imu_48,imu_49,imu_50
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
5,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [11]:
df.tail()

,activity_id,imu_0,imu_1,imu_2,imu_3,imu_4,imu_5,imu_6,imu_7,imu_8,...,imu_41,imu_42,imu_43,imu_44,imu_45,imu_46,imu_47,imu_48,imu_49,imu_50
110914,0,33.625,-7.89597,2.77467,4.87733,-7.61802,2.82523,5.17494,0.030347,-0.032241,...,-0.008660,-0.017440,0.011271,-42.7926,8.03642,45.2164,0.068204,0.745913,0.160385,0.642836
110915,0,33.625,-7.81023,2.89026,5.14764,-7.63271,2.87064,5.17485,0.077752,-0.026316,...,-0.051593,-0.026622,-0.002320,-43.5512,7.55844,44.9306,0.067938,0.746003,0.160241,0.642796
110916,0,33.625,-7.89142,2.85156,4.99260,-7.60231,2.91572,5.15962,0.072759,-0.017362,...,-0.001825,0.009911,0.008395,-43.5414,7.17214,45.0729,0.067767,0.745992,0.160180,0.642843
110917,0,33.625,-7.89142,2.85156,4.99260,-7.69197,2.99189,5.17465,0.034680,-0.032184,...,0.014073,-0.024149,-0.006850,-43.5485,7.42956,44.9308,0.067697,0.746048,0.160246,0.642769
110918,0,33.625,-7.74241,2.92560,4.91761,-7.72224,3.02227,5.12931,0.049226,-0.010066,...,0.020672,0.020502,0.010129,-43.4371,8.19647,44.9303,0.068013,0.745946,0.160265,0.642848


In [86]:
patient_all_df[columns[1:]].values

dask.array<values, shape=(nan, 51), dtype=float64, chunksize=(nan, 51)>

# Model training 

In [71]:
patient_all_pd_train, patient_all_pd_test = train_test_split_sk(patient_all_pd, 
                                                                test_size=0.3, random_state=42)


In [56]:
patient_all_pd_train.shape

(1991407, 52)

In [57]:
patient_all_pd_test.shape

(853461, 52)

In [72]:
%%time
#
#Train and test split for dask dataframe
#


import dask.dataframe as dd
patient_all_df_train = dd.from_pandas(patient_all_pd_train, npartitions = 10)
patient_all_df_train = patient_all_df_train.persist()

patient_all_df_test = dd.from_pandas(patient_all_pd_test, npartitions = 10)
patient_all_df_test = patient_all_df_test.compute()

progress(patient_all_df_train, patient_all_df_test)

CPU times: user 20.6 s, sys: 1.39 s, total: 22 s
Wall time: 6.37 s


In [58]:
%%time
#Serial executiion of sklearn building 50 trees without any parallelism
sk_rf = RF_SK(n_jobs  = 1, n_estimators=50)

%mprun sk_rf_model = sk_rf.fit(patient_all_pd_train[columns[1:]], patient_all_pd_train.activity_id)

sk_rf = RF_SK(n_jobs=-1, n_estimators=50)



CPU times: user 34min 5s, sys: 5.21 s, total: 34min 11s
Wall time: 25min 53s


In [34]:
%%time

# BUilding Random forest using Dask for 50 trees using 4 workers


# from dask_ml.model_selection import train_test_split as train_test_split_df
#patient_all_df_train, patient_all_df_test = train_test_split_df(patient_all_df[columns[1:]], patient_all_df.activity_id)
#Using 4 workers eith a total of 192 cores and 520 GB memory
#Running sklearn for 50 trees with all cores

with parallel_backend('dask'):
        dask_rf_model = sk_rf.fit(patient_all_df_train[columns[1:]], patient_all_df_train.activity_id)
progress(dask_rf_model)

CPU times: user 14 s, sys: 1.89 s, total: 15.9 s
Wall time: 1min 18s


In [35]:
%%time
# BUilding Random forest using scikit-learn using all cores

sk_rf_model = sk_rf.fit(patient_all_pd_train[columns[1:]], patient_all_pd_train.activity_id)

CPU times: user 46min 42s, sys: 4min 6s, total: 50min 49s
Wall time: 1min 29s


In [39]:
# client.replicate()

(Delayed('int-7b6bb9f6-7fdd-4cf1-a0df-f8d8b1295272'), 52)

# Making predictions

In [37]:
%%time

#
#Calulating the accuracy for the algorithm built using Dask
#
y_pred = dask_rf_model.predict(patient_all_df_test[columns[1:]])
acc_dask = accuracy_score(patient_all_df_test.activity_id, y_pred)
print(acc_dask)

0.9995711579087972
CPU times: user 37.2 s, sys: 1min 28s, total: 2min 5s
Wall time: 2.95 s


In [38]:
%%time

#
#Calculating the accuracy for the algorithm built using Sklearn
#

y_pred = sk_rf_model.predict(patient_all_pd_test[columns[1:]])
acc_sk = accuracy_score(patient_all_pd_test.activity_id, y_pred)
print(acc_sk)

0.9995711579087972
CPU times: user 55.4 s, sys: 1min 26s, total: 2min 21s
Wall time: 3.16 s


In [ ]:
%load_ext memory_profiler

# Doing some experiments with a medium sized and large custom dataset

In [4]:
from dask_ml.datasets import make_classification
from sklearn.datasets import make_classification as sk_make_classf

In [5]:
# %%timeit
X, y = sk_make_classf(n_samples=1000000, random_state=0)
X[:5]

array([[-0.26713096, -0.4882575 , -2.43008658,  0.78218964,  1.79746173,
        -0.16888515, -0.56677967,  0.29882333, -0.09893902,  1.75612743,
        -1.82722012, -0.65391374, -0.03029503, -0.82787519,  0.65285578,
         1.0543947 , -0.88524403,  1.40167589,  0.80658882, -0.64143233],
       [ 0.05807287,  0.99528919,  0.12051257, -0.76374877, -0.68595339,
         2.20630364,  0.02161348,  0.37515075, -0.19961594, -0.32758905,
        -0.93350546, -1.03831718,  0.21978325, -1.27490547,  0.14761016,
         1.37749136,  0.11472287, -0.31900551,  1.24403302, -1.49077402],
       [-0.23482836,  0.54239715,  0.03364741, -0.38204206, -1.43249441,
        -0.53892947,  1.03913615, -1.00239419,  0.56471407, -0.4381485 ,
         1.92879222, -2.27579505, -0.82915498,  1.29838157, -0.11506946,
         0.43504248, -0.07223782,  0.94305017,  1.32051199,  0.12346298],
       [-0.81819573, -1.08520897,  0.10609962, -0.23298968,  0.05860691,
        -0.6041117 , -0.91473755, -1.12212224,  

In [9]:
# %%timeit
X_ds, y_ds = make_classification(n_samples=1000000, random_state = 1, chunks= 1000000/8)
X_ds[:5]

dask.array<getitem, shape=(5, 20), dtype=float64, chunksize=(5, 20)>

In [6]:
# %%memit 
import sys
sys.getsizeof(X)

160000112

In [10]:
%%time
with parallel_backend('dask'):
        dask_rf_model = sk_rf.fit(X_ds, y_ds)
progress(dask_rf_model)

CPU times: user 12.5 s, sys: 947 ms, total: 13.4 s
Wall time: 47.1 s


In [11]:
%%time
sk_rf_model = sk_rf.fit(X, y)

CPU times: user 17min 3s, sys: 3min 47s, total: 20min 50s
Wall time: 35.8 s


# Persisting dataset with 1 million data points

In [81]:
#
#Genenrating new data points
#

def get_new_data(no_samples):
    
    X_ds, y_ds = make_classification(n_samples = no_samples, random_state = 1, chunks= no_samples/8)
    X_ds = client.persist(X_ds)
    y_ds = client.persist(y_ds)
    X, y = sk_make_classf(n_samples=no_samples, random_state=0)
    
    return(X_ds, y_ds, X, y)

In [82]:
client.cancel(X_ds)
client.cancel(y_ds)
del X_ds
del y_ds

NameError: name 'X_ds' is not defined

In [83]:
#
# Creating 1 million data points
#

X_ds, y_ds, X, y = get_new_data(1000000)

In [84]:
%%time

#
#Fitting using dask
#

with parallel_backend('dask'):
        dask_rf_model = sk_rf.fit(X_ds, y_ds)
progress(dask_rf_model)

CPU times: user 3.61 s, sys: 7.33 s, total: 10.9 s
Wall time: 40.9 s


In [85]:
%%time

#
#Fitting with sci-kit learn
#

sk_rf_model = sk_rf.fit(X, y)

CPU times: user 17min 48s, sys: 2min 29s, total: 20min 17s
Wall time: 35.3 s


# 15 million data points

In [ ]:
client.cancel(X_ds)
client.cancel(y_ds)
del X_ds
del y_ds

In [ ]:
#
#Creating 15 million data points
#

X_ds, y_ds, X, y = get_new_data(15000000)

In [26]:
%%time

#
# Model fitting using dask
#

with parallel_backend('dask'):
        dask_rf_model = sk_rf.fit(X_ds, y_ds)
progress(dask_rf_model)

CPU times: user 31.3 s, sys: 16.2 s, total: 47.4 s
Wall time: 7min 45s


In [64]:
%%time

#
#Model fitting using scikit-learn
#

sk_rf_model = sk_rf.fit(X, y)

CPU times: user 6h 14min 9s, sys: 5min 18s, total: 6h 19min 28s
Wall time: 11min 55s


# Grid Search Using Dask and scikit-learn

In [76]:
%%time

#
#Hyper-parameter tuning using dask
#


from sklearn.externals.joblib import parallel_backend
with parallel_backend('dask'):
    # Create the parameter grid based on the results of random search 
    param_grid = {
    'bootstrap': [True],
    'max_depth': [8, 9],
#     'max_features': [2, 3],
#     'min_samples_leaf': [4, 5],
#     'min_samples_split': [8, 10],
    'n_estimators': [10, 20]
    }
    # Create a based model
    from sklearn.ensemble import RandomForestClassifier
    rf = RandomForestClassifier(n_jobs=-1)
    import dask_searchcv as dcv
    grid_search = dcv.GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3)
    grid_search.fit(patient_all_pd[columns[1:]],patient_all_pd.activity_id)
    
progress(grid_search)

/N/u/ronashah/.conda/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/N/u/ronashah/.conda/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/N/u/ronashah/.conda/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/N/u/ronashah/.conda/envs/py36/lib/python3.6/site-packages/sklearn/ensemble/base.py:154: UserWarning: Loky-backed parallel loops cannot be nested below threads, setting n_jobs=1
  n_jobs = min(effective_n_jobs(n_jobs), n_estimators)
/N/u/ronashah/.conda/envs/py36/lib/python3.6/site-packages/sklearn/e

CPU times: user 1h 12min 51s, sys: 30min 54s, total: 1h 43min 45s
Wall time: 2min 33s


In [77]:
%%time

#
#Hyper-parameter tuning using scikit-learn
#


from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs = -1)
param_grid = {s
'bootstrap': [True],
'max_depth': [8, 9],
# 'max_features': [2, 3],
# 'min_samples_leaf': [4, 5],
# 'min_samples_split': [8, 10],
'n_estimators': [10, 20]
}
grid_search_sk = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3)
grid_search_sk.fit(patient_all_pd[columns[1:]],patient_all_pd.activity_id)


CPU times: user 8min 55s, sys: 13min 46s, total: 22min 41s
Wall time: 4min 3s
